## Data Wrangling

In [2]:
import pandas as pd
import bs4
import requests
import requests
import seaborn as sns
import numpy as np
import re
from pathlib import Path
import json

from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

## Step 1: obtain data and metadata

### Topic: Fuel consumption ratings (Statistics Canada)

Site https://open.canada.ca/data/en/dataset/98f1a129-f628-4ce4-b24d-6f16bf24dd64

In [3]:
url_open_canada = "https://open.canada.ca/data/api/action/package_show?id=98f1a129-f628-4ce4-b24d-6f16bf24dd64"

json_resp = requests.get(url_open_canada)

In [4]:
json_resp.headers.get('Content-Type','')

'application/json;charset=utf-8'

In [5]:
if json_resp.status_code == 200 and 'application/json' in json_resp.headers.get('Content-Type',''):
    open_canada_data = json_resp.json()
else:
    print("Error")

In [6]:
open_canada_data.keys()

dict_keys(['help', 'success', 'result'])

In [7]:
# Data wrangling
data_entries = pd.json_normalize(open_canada_data['result'], record_path="resources")
data_entries['language'] = data_entries['language'].apply(lambda col: col[0])
data_entries_english = data_entries[data_entries['language']=='en']


In [28]:
assert url_content is not None

#### Automate data extraction

In [9]:
model_dict = {"4WD/4X4":"Four-wheel drive",
	      "AWD": "All-wheel drive",
	      "FFV": "Flexible-fuel vehicle",
	      "SWB": "Short wheelbase",
	      "LWB" : "Long wheelbase",
	      "EWB" : "Extended wheelbase",
	      "CNG" : "Compressed natural gas",
	      "NGV" : "Natural gas vehicle",
	      "#" : "High output engine that provides more power than the standard engine of the same size"
 }

transmission_dict = {"A": "automatic",
		     "AM": "automated manual",
		     "AS": "automatic with select Shift",
		     "AV": "continuously variable",
		     "M": "manual",
		     "1 – 10" : "Number of gears",

}


fuel_dict = {"X": "regular gasoline",
	     "Z": "premium gasoline",
 	     "D": "diesel",
	     "E": "ethanol (E85)",
	     "N": "natural gas",
	     "B": "electricity"
	
}

In [10]:
def convert_model_key_words(s, dictionary):
    """Add columns from footnote"""

    group = "unspecified"
    for key in dictionary:
        if key in s:
            group = dictionary[key]
            break
    return group

,cache_last_updated,unique_identifier,package_id,datastore_contains_all_records_of_source_file,validation_status,datastore_active,character_set,validation_timestamp,id,state,...,language,created,url,last_modified,resource_type,position,revision_id,data_quality,name_translated.fr,name_translated.en
0,None,,98f1a129-f628-4ce4-b24d-6f16bf24dd64,False,,False,,,026e45b4-eb63-451f-b34f-d9308ea3a3d9,active,...,en,2017-03-31T09:40:17.244058,https://www.nrcan.gc.ca/sites/nrcan/files/oee/...,None,dataset,0,ffd5fc97-9248-48ed-b1bc-7fa542cdad40,[],Véhicules électriques à batterie 2012-2022 (20...,Battery-electric vehicles 2012-2022 (2022-05-16)
2,None,,98f1a129-f628-4ce4-b24d-6f16bf24dd64,False,,False,,,8812228b-a6aa-4303-b3d0-66489225120d,active,...,en,2017-03-31T09:40:17.244000,https://www.nrcan.gc.ca/sites/nrcan/files/oee/...,None,dataset,2,28104dc6-6147-4ecc-913c-ad13f4f565bf,[],Véhicules hybrides électriques rechargeables 2...,Plug-in hybrid electric vehicles 2012-2022 (20...
4,None,,98f1a129-f628-4ce4-b24d-6f16bf24dd64,False,,False,,,87fc1b5e-fafc-4d44-ac52-66656fc2a245,active,...,en,2022-02-04T22:21:36.653608,https://www.nrcan.gc.ca/sites/nrcan/files/oee/...,None,dataset,4,28440c56-0108-4a36-8d25-c092ea07889a,[],Cotes de consommation de carburant 2022 (2022-...,2022 Fuel Consumption Ratings (2022-08-18)
6,None,,98f1a129-f628-4ce4-b24d-6f16bf24dd64,False,,False,,,f2e53a2b-e075-473a-9a9c-5d7bef68d07d,active,...,en,2021-02-03T00:31:03.706926,https://www.nrcan.gc.ca/sites/nrcan/files/oee/...,None,dataset,6,e5e595b7-f949-4492-ab9a-9bf3207e90b1,[],Cotes de consommation de carburant 2021 (2022-...,2021 Fuel Consumption Ratings (2022-08-09)
8,None,,98f1a129-f628-4ce4-b24d-6f16bf24dd64,False,NaN,False,,NaN,56a89c09-d609-41cd-8838-9dd9905d3cfc,active,...,en,2020-01-29T14:40:52.639636,https://www.nrcan.gc.ca/sites/nrcan/files/oee/...,None,dataset,8,5ec3d0f3-5d9d-4c6e-a3fd-78e62514c945,[],Cotes de consommation de carburant 2020 (2021-...,2020 Fuel Consumption Ratings (2021-09-29)
10,None,,98f1a129-f628-4ce4-b24d-6f16bf24dd64,False,NaN,False,,NaN,e61e33a6-3522-43c7-b3ef-0128a05e87c7,active,...,en,2019-01-29T15:41:33.769052,https://www.nrcan.gc.ca/sites/nrcan/files/oee/...,None,dataset,10,5ec3d0f3-5d9d-4c6e-a3fd-78e62514c945,[],Cotes de consommation de carburant 2019 (2021-...,2019 Fuel Consumption Ratings (2021-09-29)
12,None,,98f1a129-f628-4ce4-b24d-6f16bf24dd64,False,NaN,False,,NaN,ccd37443-63f3-4e94-90ef-0f25d5020edd,active,...,en,2017-12-21T23:00:11.211372,https://www.nrcan.gc.ca/sites/nrcan/files/oee/...,None,dataset,12,5ec3d0f3-5d9d-4c6e-a3fd-78e62514c945,[],Cotes de consommation de carburant 2018 (2021-...,2018 Fuel Consumption Ratings (2021-09-29)
14,None,,98f1a129-f628-4ce4-b24d-6f16bf24dd64,False,,False,,,522929ff-1ee6-4bae-b583-7ac70e921ec3,active,...,en,2017-03-31T09:40:17.243947,https://www.nrcan.gc.ca/sites/nrcan/files/oee/...,None,dataset,14,1be0d323-bada-4465-bdec-a0ba285d28d8,[],Cotes de consommation de carburant 2017 (2020-...,2017 Fuel Consumption Ratings (2020-03-17)
16,None,,98f1a129-f628-4ce4-b24d-6f16bf24dd64,False,,False,,,6f6e681b-630e-4c4d-a18d-124801aa952c,active,...,en,2017-03-31T09:40:17.243891,https://www.nrcan.gc.ca/sites/nrcan/files/oee/...,None,dataset,16,76e679d5-a36f-456d-966e-7e10d8d3d298,[],Cotes de consommation de carburant 2016 (2020-...,2016 Fuel Consumption Ratings (2020-03-17)
18,None,,98f1a129-f628-4ce4-b24d-6f16bf24dd64,False,,False,,,332be680-9577-42c6-8c47-a0380ef48c5e,active,...,en,2017-03-31T09:40:17.243835,https://www.nrcan.gc.ca/sites/nrcan/files/oee/...,None,dataset,18,76e679d5-a36f-456d-966e-7e10d8d3d298,[],Cotes de consommation de carburant 2015 (2020-...,2015 Fuel Consumption Ratings (2020-03-17)


In [30]:
data_entries_english['url'].to_list()

['https://www.nrcan.gc.ca/sites/nrcan/files/oee/files/csv/MY2012-2022%20Battery%20Electric%20Vehicles.csv',
 'https://www.nrcan.gc.ca/sites/nrcan/files/oee/files/csv/MY2012-2022%20Plug-in%20Hybrid%20Electric%20Vehicles.csv',
 'https://www.nrcan.gc.ca/sites/nrcan/files/oee/files/csv/MY2022%20Fuel%20Consumption%20Ratings.csv',
 'https://www.nrcan.gc.ca/sites/nrcan/files/oee/files/csv/MY2021%20Fuel%20Consumption%20Ratings.csv',
 'https://www.nrcan.gc.ca/sites/nrcan/files/oee/files/csv/MY2020%20Fuel%20Consumption%20Ratings.csv',
 'https://www.nrcan.gc.ca/sites/nrcan/files/oee/files/csv/MY2019%20Fuel%20Consumption%20Ratings.csv',
 'https://www.nrcan.gc.ca/sites/nrcan/files/oee/files/csv/MY2018%20Fuel%20Consumption%20Ratings.csv',
 'https://www.nrcan.gc.ca/sites/nrcan/files/oee/files/csv/MY2017%20Fuel%20Consumption%20Ratings.csv',
 'https://www.nrcan.gc.ca/sites/nrcan/files/oee/files/csv/MY2016%20Fuel%20Consumption%20Ratings.csv',
 'https://www.nrcan.gc.ca/sites/nrcan/files/oee/files/csv/MY2

In [40]:
folder = "C:/Users/Laura GF/Documents/GitHub/machine-learning-capstone/data/"
fuel_based_df = []
electric_based_df = []
total_no_records = 0
for item in data_entries_english[['name','url']].iterrows():
    # Form file name
    file_name = f'{item[1]["name"].replace(" ","_")}.csv'
    # Perform query
    csv_req = requests.get(item[1]['url'])
    # Parse content
    url_content = csv_req.content
    
    # Save content into file
    csv_file = open(Path(folder + "raw-data",file_name), 'wb',)
    csv_file.write(url_content)
    csv_file.close()
    
    # Data cleaning
    df = pd.read_csv(Path(folder + "raw-data",file_name), sep=",", low_memory=False, encoding='cp1252')
    sample_df_col = df.dropna(thresh=1 ,axis=1).dropna(thresh=1 ,axis=0)
    sample_df_col.columns = [item.lower() for item in sample_df_col.columns]
    footer = sample_df_col[(sample_df_col['make'].str.contains("=")) & ~(sample_df_col['make'].isna())]
    sample_df_no_footer = sample_df_col.dropna(thresh=3 ,axis=0)
    
    # Remove Unnamed cols
    cols = sample_df_no_footer.columns
    cleaned_cols = [re.sub(r'unnamed: \d*', "fuel consumption", item) if "unnamed" in item else item for item in cols]


    # Clean row 1 on df
    str_item_cols = [str(item) for item in sample_df_no_footer.iloc[0:1,].values[0]]
    str_non_nan = ["" if item=='nan' else item for item in str_item_cols]

    # Form new columns
    new_cols = []
    for itema,itemb in zip(cleaned_cols, str_non_nan):
        new_cols.append(f'{itema}_{itemb}'.lower().replace("*","").replace(" ","").replace(r'#=highoutputengine',""))


    final_df = sample_df_no_footer.iloc[1:, ].copy()
    final_df.columns = new_cols
    
    # Save clean df
    print("Number of records in file", file_name, ":", final_df.size)
    total_no_records += final_df.size
    
    # Populate dataframe with information from the footnotes
    if "electric" in item[1]['name']:
        final_df["type_of_wheel_drive"] = final_df['model.1_'].apply(lambda x: convert_model_key_words(x, model_dict)) 
        final_df["type_of_transmission"] = final_df['transmission_'].apply(lambda x: convert_model_key_words(x, transmission_dict)) 
        electric_based_df.append(final_df)
        final_df.to_csv(Path(folder + "clean-data",f'{file_name}'))
        
        continue
    
    else:
        final_df["type_of_wheel_drive"] = final_df['model.1_'].apply(lambda x: convert_model_key_words(x, model_dict)) 
        final_df["type_of_transmission"] = final_df['transmission_'].apply(lambda x: convert_model_key_words(x, transmission_dict)) 
        final_df["type_of_fuel"] = final_df['fuel_type'].apply(lambda x: convert_model_key_words(x, fuel_dict)) 
        fuel_based_df.append(final_df)
        final_df.to_csv(Path(folder + "clean-data",f'{file_name}'))
        
master_fuel_cons = pd.concat(fuel_based_df)

master_fuel_cons.to_csv(Path(folder + "clean-data","1995_2022_Fuel_Consumption_Ratings.csv"))

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


TypeError: argument should be integer or bytes-like object, not 'str'

### Topic: Number of cars sold in Canada by make and model

Site: https://www.goodcarbadcar.net/


The data was extracted using Scrapy

In [ ]:
raw_data = f'{folder}raw-data/'
clean_data = f'{folder}clean-data/'

# Read data
json_2021_df = pd.read_json(Path(raw_data,"2021_canada_vehicle_sales.json"))
json_2021_df.dropna(how="all", inplace=True)
json_2021_df['Year'] = 2021

json_2020_df = pd.read_json(Path(raw_data,"2020_canada_vehicle_sales.json"))
json_2020_df.dropna(how="all", inplace=True)
json_2020_df['Year'] = 2020

json_2019_df = pd.read_json(Path(raw_data,"2019_canada_vehicle_sales.json"))
json_2019_df.dropna(how="all", inplace=True)
json_2019_df['Year'] = 2019

In [ ]:
json_2021_df

### Topic: New motor vehicle registrations, quarterly (Statistics Canada)

Site https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=2010002401



In [ ]:
vehicle_registrations = "https://www150.statcan.gc.ca/n1/tbl/csv/20100024-eng.zip"
resp = urlopen(vehicle_registrations)
myzip = ZipFile(BytesIO(resp.read()))
file_name = [item for item in myzip.namelist() if "MetaData" not in item]

In [ ]:
vehicle_reg_csv = myzip.open(file_name[0])
vehicle_reg_df = pd.read_csv(vehicle_reg_csv)

In [ ]:
vehicle_reg_df.drop(columns=['DGUID',
                             'UOM_ID',
                             'SCALAR_ID',
                             'VECTOR',
                             'COORDINATE',
                             'STATUS',
                             'SYMBOL',
                             'TERMINATED',
                             'DECIMALS'], inplace=True)

In [ ]:
vehicle_reg_df.to_csv(Path(folder+"clean-data", "new_motor_vehicle_reg.csv"))

In [ ]:
vehicle_reg_df

### Topic: New zero-emission vehicle registrations, quarterly (Statistics Canada)

https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=2010002501

In [ ]:
near_zero_vehicle = "https://www150.statcan.gc.ca/n1/tbl/csv/20100025-eng.zip"

resp = urlopen(near_zero_vehicle)
myzip = ZipFile(BytesIO(resp.read()))
file_name = [item for item in myzip.namelist() if "MetaData" not in item]

In [ ]:
near_zero_vehicle_reg_csv = myzip.open(file_name[0])
near_zero_vehicle_reg_df = pd.read_csv(near_zero_vehicle_reg_csv)

In [ ]:
near_zero_vehicle_reg_df.drop(columns=['DGUID',
                                       'UOM_ID',
                                       'SCALAR_ID',
                                       'VECTOR',
                                       'COORDINATE',
                                       'STATUS',
                                       'SYMBOL','TERMINATED','DECIMALS'], inplace=True)

In [ ]:
near_zero_vehicle_reg_df.to_csv(Path(folder+"clean-data", "near_zero_vehicle_registrations.csv"))

In [ ]:
near_zero_vehicle_reg_df

### Topic: Sales of fuel used for road motor vehicles, annual (Statistics Canada)

https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=2310006601

In [ ]:
fuel_sold = "https://www150.statcan.gc.ca/n1/tbl/csv/23100066-eng.zip"

resp = urlopen(fuel_sold)
myzip = ZipFile(BytesIO(resp.read()))
file_name = [item for item in myzip.namelist() if "MetaData" not in item]

fuel_sold_csv = myzip.open(file_name[0])
fuel_sold_df = pd.read_csv(fuel_sold_csv)

fuel_sold_df.drop(columns=['DGUID',
                                       'UOM_ID',
                                       'SCALAR_ID',
                                       'VECTOR',
                                       'COORDINATE',
                                       'STATUS',
                                       'SYMBOL',
                                       'TERMINATED',
                                       'DECIMALS'], inplace=True)

fuel_sold_df.to_csv(Path(folder+"clean-data", "fuel_sold_motor_vehicles.csv"))

In [ ]:
fuel_sold_df

### Topic: Vehicle registrations, by type of vehicle (Statistics Canada)

https://open.canada.ca/data/en/dataset/9aea572f-f54f-42a1-b411-0b06390ed9f9

In [ ]:
vehicle_reg = "https://www150.statcan.gc.ca/n1/tbl/csv/23100067-eng.zip"

resp = urlopen(vehicle_reg)
myzip = ZipFile(BytesIO(resp.read()))
file_name = [item for item in myzip.namelist() if "MetaData" not in item]

vehicle_reg_csv = myzip.open(file_name[0])
vehicle_reg_df = pd.read_csv(vehicle_reg_csv)

vehicle_reg_df.drop(columns=['DGUID',
                                       'UOM_ID',
                                       'SCALAR_ID',
                                       'VECTOR',
                                       'COORDINATE',
                                       'STATUS',
                                       'SYMBOL',
                                       'TERMINATED',
                                       'DECIMALS'], inplace=True)



In [ ]:
vehicle_reg_df.to_csv(Path(folder+"clean-data", "vehicle_registrations_type_vehicle.csv"))